In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Huntingtons_Disease"
cohort = "GSE34721"

# Input paths
in_trait_dir = "../../input/GEO/Huntingtons_Disease"
in_cohort_dir = "../../input/GEO/Huntingtons_Disease/GSE34721"

# Output paths
out_data_file = "../../output/preprocess/Huntingtons_Disease/GSE34721.csv"
out_gene_data_file = "../../output/preprocess/Huntingtons_Disease/gene_data/GSE34721.csv"
out_clinical_data_file = "../../output/preprocess/Huntingtons_Disease/clinical_data/GSE34721.csv"
json_path = "../../output/preprocess/Huntingtons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Continuous analysis captures cellular states that reflect dominant effects of the HTT CAG repeat in human lymphoblastoid cell lines."
!Series_summary	"In Huntington’s disease (HD), expanded HTT CAG repeat length correlates strongly with age at motor onset, indicating that it determines the rate of the disease process leading to diagnostic clinical manifestations. Similarly, in normal individuals, HTT CAG repeat length is correlated with biochemical differences that reveal it as a functional polymorphism.  Here, we tested the hypothesis that gene expression signatures can capture continuous, length-dependent effects of the HTT CAG repeat.  Using gene expression datasets for 107 HD and control lymphoblastoid cell lines, we constructed mathematical models in an iterative manner, based upon CAG correlated gene expression patterns in randomly chosen training samples, and tested their predictive power in test samples.  Predicted CAG repeat lengths were 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# from lymphoblastoid cell lines with varying HTT CAG repeat lengths
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Huntington's Disease) is determined by HTT CAG repeat length
# In the context of HD, CAG repeat length > 35 is considered pathogenic
trait_row = 1  # The HTT CAG repeat length is in row 1

# Age data is not available in the sample characteristics
age_row = None

# Gender data is available in row 0
gender_row = 0

# 2.2 Data Type Conversion

# Function to convert HTT CAG repeat length to binary trait (HD status)
def convert_trait(value):
    try:
        # Extract the number after the colon
        if isinstance(value, str) and ":" in value:
            cag_str = value.split(":")[1].strip()
            cag_length = int(cag_str)
            # In Huntington's Disease, CAG repeat length > 35 is pathogenic
            return 1 if cag_length >= 36 else 0
        return None
    except:
        return None

# No age conversion function needed as age data is not available
convert_age = None

# Function to convert gender to binary (0 for female, 1 for male)
def convert_gender(value):
    try:
        if isinstance(value, str) and ":" in value:
            gender = value.split(":")[1].strip().lower()
            if "female" in gender:
                return 0
            elif "male" in gender:
                return 1
        return None
    except:
        return None

# 3. Save Metadata
# Since trait_row is not None, trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
# First, load clinical data (assuming it's available from a previous step)
try:
    # Use the geo_select_clinical_features function to extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,  # This should be defined in a previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("Preview of extracted clinical features:")
    print(preview_df(clinical_features))
    
    # Save clinical data to the specified output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
except NameError:
    print("Clinical data not available from previous steps or not properly defined.")


Preview of extracted clinical features:
{'GSM853700': [1.0, 1.0], 'GSM853701': [0.0, 0.0], 'GSM853702': [0.0, 1.0], 'GSM853703': [1.0, 0.0], 'GSM853704': [1.0, 1.0], 'GSM853705': [1.0, 1.0], 'GSM853706': [0.0, 1.0], 'GSM853707': [1.0, 1.0], 'GSM853708': [1.0, 0.0], 'GSM853709': [0.0, 0.0], 'GSM853710': [0.0, 0.0], 'GSM853711': [1.0, 0.0], 'GSM853712': [1.0, 0.0], 'GSM853713': [0.0, 1.0], 'GSM853714': [1.0, 1.0], 'GSM853715': [1.0, 0.0], 'GSM853716': [0.0, 1.0], 'GSM853717': [0.0, 0.0], 'GSM853718': [1.0, 1.0], 'GSM853719': [1.0, 1.0], 'GSM853720': [1.0, 0.0], 'GSM853721': [1.0, 1.0], 'GSM853722': [1.0, 0.0], 'GSM853723': [1.0, 0.0], 'GSM853724': [1.0, 0.0], 'GSM853725': [1.0, 0.0], 'GSM853726': [1.0, 1.0], 'GSM853727': [1.0, 1.0], 'GSM853728': [1.0, 1.0], 'GSM853729': [1.0, 0.0], 'GSM853730': [1.0, 0.0], 'GSM853731': [1.0, 1.0], 'GSM853732': [1.0, 0.0], 'GSM853733': [1.0, 1.0], 'GSM853734': [1.0, 0.0], 'GSM853735': [1.0, 0.0], 'GSM853736': [1.0, 1.0], 'GSM853737': [1.0, 0.0], 'GSM85373

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Huntingtons_Disease/GSE34721/GSE34721_series_matrix.txt.gz


Gene data shape: (54675, 227)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing gene identifiers in the gene expression data
# The identifiers shown (e.g., '1007_s_at', '1053_at', etc.) appear to be Affymetrix probe IDs,
# not standard human gene symbols like BRCA1, TP53, etc.
# These probe IDs need to be mapped to official gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# Based on the gene annotation dataframe, the mapping between probe IDs and gene symbols is:
# 'ID' column: Contains the probe IDs that match the gene expression data index
# 'Gene Symbol' column: Contains the gene symbols we want to map to

# 1. Create the mapping dataframe using the get_gene_mapping function
prob_col = 'ID'  # Column that contains the probe IDs
gene_col = 'Gene Symbol'  # Column that contains the gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Print mapping information
print(f"Mapping dataframe created with shape: {gene_mapping.shape}")
print("First few rows of mapping dataframe:")
print(gene_mapping.head())

# 2. Apply the gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the mapped gene expression data
print(f"\nGene-level expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# 3. Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nNormalized gene expression data shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe created with shape: (45782, 2)
First few rows of mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A



Gene-level expression data shape: (21278, 227)
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')

Normalized gene expression data shape: (19845, 227)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Huntingtons_Disease/gene_data/GSE34721.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Gene data shape after loading: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# 2. Load clinical data from the saved file
print("\nLoading clinical data...")
clinical_data = pd.read_csv(out_clinical_data_file)
print(f"Clinical data shape: {clinical_data.shape}")
print("Clinical data preview:")
print(preview_df(clinical_data))

# Create a proper DataFrame structure with samples as index (rows) and features as columns
# First, get sample IDs from the columns
sample_ids = clinical_data.columns.tolist()

# Then create a proper clinical DataFrame
proper_clinical_df = pd.DataFrame({
    trait: clinical_data.iloc[0].values,
    'Gender': clinical_data.iloc[1].values
}, index=sample_ids)

print("\nRestructured clinical data:")
print(preview_df(proper_clinical_df))

# 3. Link clinical and gene data properly
print("\nLinking clinical and genetic data...")
linked_data = pd.merge(
    proper_clinical_df, 
    gene_data.T, 
    left_index=True, 
    right_index=True
)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This GSE34721 dataset contains gene expression data from lymphoblastoid cell lines with varying HTT CAG repeat lengths, relevant to Huntington's Disease."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Normalizing gene symbols...


Gene data shape after loading: (19845, 227)
First 10 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')

Loading clinical data...
Clinical data shape: (2, 227)
Clinical data preview:
{'GSM853700': [1.0, 1.0], 'GSM853701': [0.0, 0.0], 'GSM853702': [0.0, 1.0], 'GSM853703': [1.0, 0.0], 'GSM853704': [1.0, 1.0], 'GSM853705': [1.0, 1.0], 'GSM853706': [0.0, 1.0], 'GSM853707': [1.0, 1.0], 'GSM853708': [1.0, 0.0], 'GSM853709': [0.0, 0.0], 'GSM853710': [0.0, 0.0], 'GSM853711': [1.0, 0.0], 'GSM853712': [1.0, 0.0], 'GSM853713': [0.0, 1.0], 'GSM853714': [1.0, 1.0], 'GSM853715': [1.0, 0.0], 'GSM853716': [0.0, 1.0], 'GSM853717': [0.0, 0.0], 'GSM853718': [1.0, 1.0], 'GSM853719': [1.0, 1.0], 'GSM853720': [1.0, 0.0], 'GSM853721': [1.0, 1.0], 'GSM853722': [1.0, 0.0], 'GSM853723': [1.0, 0.0], 'GSM853724': [1.0, 0.0], 'GSM853725': [1.0, 0.0], 'GSM853726': [1.0, 1.0], 'GSM853727': [1.0, 1.0], '

Linked data shape after handling missing values: (227, 19847)

Checking for bias in dataset features...
For the feature 'Huntingtons_Disease', the least common label is '0.0' with 77 occurrences. This represents 33.92% of the dataset.
The distribution of the feature 'Huntingtons_Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 112 occurrences. This represents 49.34% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Huntingtons_Disease/GSE34721.csv
